# Raster data processing
This notebook is designed to download and process raster-based environmental data into consistent gridded datasets for analysis in geospatial software and in MaxEnt species distribution modeling.

For each data type, we run the following steps:
- downloading from a remote server
- handling no-data values
- mosaicing and reprojecting imagery
- resampling to multiple grid sizes
- converting to maxent's raster data format

The raw data are hosted on a publicly-accessible cloud bucket, `gs://aedes-americas/`. Some of the data processed in this notebook will be uploaded to this bucket. May of the raw data were generated from Google Earth Engine, and the scripts used to generate these data are available in the directory `aedes-americas/data-processing/earthengine/`.

To run this notebook, first navigate to the root directory of the `aedes-americas` repository and activate the conda environment.

```bash
cd /path/to/aedes-americas
conda activate aedes
jupyter notebook
```

## 0.1 Loading modules and configs

In [1]:
# set to allow module re-loading during development
%load_ext autoreload
%autoreload 2

In [42]:
# import packages
import os
import gdal
import glob
import yaml
import numpy as np
import otbApplication as otb

# raise gdal runtime errors
gdal.UseExceptions()

# set up orfeo application calls
bandMath = otb.Registry.CreateApplication("BandMath")
manageNoData = otb.Registry.CreateApplication("ManageNoData")

# set the otb creation options to pass to output tif files
otb_creation_options = "&gdal:co:COMPRESS=DEFLATE&gdal:co:TILED=YES"
gdal_creation_options = ["COMPRESS=DEFLATE", "TILED=YES"]

In [43]:
# read the config into memory
with open('../config.yml', 'r+') as f:
    config = yaml.load(f, Loader=yaml.Loader)

In [34]:
# create the data directories if they don't yet exist
for directory in [config['data-dir'], config['data-mx-in'], config['data-mx-out']]:
    if not os.path.exists(directory):
        print('Creating data directory: {}'.format(directory))
        os.mkdir(directory)

grid_dirs = list()
for grid_size in config['grid-size']:
    directory = os.path.join(config['data-mx-in'], "{:06d}-m".format(int(grid_size)))
    grid_dirs.append(directory)
    if not os.path.exists(directory):
        print('Creating data directory: {}'.format(directory))
        os.mkdir(directory)

Creating data directory: /external/aedes-americas/data-mx-in/001000-m
Creating data directory: /external/aedes-americas/data-mx-in/005000-m
Creating data directory: /external/aedes-americas/data-mx-in/010000-m
Creating data directory: /external/aedes-americas/data-mx-in/050000-m
Creating data directory: /external/aedes-americas/data-mx-in/100000-m


### 0.2 helper functions for common tasks

In [ ]:
# updating no-data values
def update_nodata(path, nodata=config['nodataata']):
    
    # read the file refGA_ReadOnlyeadOnlyeadOnlyadOnlyadOnlyget band count
    ref = gdal.Open(path, gdal.GA_Update)
    n_bands = ref.RasterCount
    
    # replace no-data values for each band
    for band_number in range(n_bands):
        band = ref.GetRasterBand(band_number + 1)
        band.SetNoDataValue(nodata)
        band.FlushCache()
    
    # write these changes to disk
    ref.FlushCache()
    ref = None
    return True

## 1.1 Population density data
The human population density data were acquired via WorldPop (https://www.worldpop.org/) and accessed via earthengine.

In [35]:
# set the name key
key = 'population'

### 1.2 Downloading data

### 1.3 Handling no-data values

In [15]:
# get the list of raw file paths
raw_paths = glob.glob(os.path.join(config['data-dir'], 'LACR-pop-density-*'))
raw_paths.sort()

In [23]:
# replace all the `nan` no-data values with numeric no-data values
for input_path in raw_paths:
    
    # set the output file name
    output_path = '{file}?{options}'.format(file = input_path[:-4]+"-nd.tif", options = otb_creation_options)
    
    # build the otb command
    manageNoData.SetParameterString("in", input_path)
    manageNoData.SetParameterString("out", output_path)
    manageNoData.SetParameterString("mode", "changevalue")
    manageNoData.SetParameterValue("mode.changevalue.newv", config['nodata'])
    manageNoData.SetParameterValue("usenan", True)
    
    # run it
    manageNoData.ExecuteAndWriteOutput()
    
    # sanitize the otb call for future use
    manageNoData.ClearValue("in")
    manageNoData.ClearValue("out")
    manageNoData.ClearValue("mode")
    manageNoData.ClearValue("mode.changevalue.newv")
    manageNoData.ClearValue("usenan")

Replacing no-data for: /external/aedes-americas/data-raw/LACR-pop-density-2015-100m-0000098304-0000065536.tif
2020-05-31 17:39:12 (INFO) ManageNoData: Default RAM limit for OTB is 256 MB
2020-05-31 17:39:12 (INFO) ManageNoData: GDAL maximum cache size is 801 MB
2020-05-31 17:39:12 (INFO) ManageNoData: OTB will use at most 8 threads
2020-05-31 17:39:13 (INFO): Estimated memory for full processing: 426.533MB (avail.: 256 MB), optimal image partitioning: 2 blocks
2020-05-31 17:39:13 (INFO): File /external/aedes-americas/data-raw/LACR-pop-density-2015-100m-0000098304-0000065536-nd.tif will be written in 3 blocks of 10752x1168 pixels
Writing /external/aedes-americas/data-raw/LACR-pop-density-2015-100m-0000098304-0000065536-nd.tif?&gdal:co:COMPRESS=DEFLATE&gdal:co:TILED=YES...: 100% [**************************************************] (2s)
Replacing no-data for: /external/aedes-americas/data-raw/LACR-pop-density-2015-100m-0000065536-0000065536.tif
2020-05-31 17:39:15 (INFO): Estimated memory

### 1.4 Mosaic and reproject the imagery
The population density data are tricky--we want to compute the sum of the population in each grid cell. However, there are no resampling tools that do that. (well, gdal 3.1 says it does, but it doesn't suppor that in it's python bindings, and the package conflicts with orfeo, so we're SOL if we want to use both)

So, my solution is to compute the average population of each 1 km grid cell then scale that by the number of valid 100m grid cells in each 1km grid cell. Since there are 10x10 100m cells per 1km cell, we'll ultimately calculate `average population * (% valid cells * 100)`.

In [ ]:
#### 1.4.1 computing average population density

In [28]:
# get the list of masked file paths
nd_paths = glob.glob(os.path.join(config['data-dir'], 'LACR-pop-density-*-nd.tif'))
nd_paths.sort()

In [50]:
# create the VRT
vrt_path = os.path.join(config['data-dir'], 'Population-data-raw.vrt')
vrt = gdal.BuildVRT(vrt_path, nd_paths)
vrt.FlushCache()

In [51]:
# set the output path
average_path = os.path.join(config['data-dir'], "Population-data-average.tif")

# set gdal warp options
warp_options = gdal.WarpOptions(
    resampleAlg = gdal.GRA_Average,
    xRes = config['grid-size'][0],
    yRes = config['grid-size'][0],
    dstSRS = 'EPSG:{}'.format(config['epsg']),
    outputBounds = config['bounds'],
    creationOptions = gdal_creation_options,
    multithread = True,
    format = 'GTiff',
    srcNodata = config['nodata'],
    dstNodata = config['nodata']
)

# run the operation
gdal.Warp(
    average_path,
    vrt_path,
    options = warp_options
)

AttributeError: 'NoneType' object has no attribute 'flushCache'

#### 1.4.2 computing the % valid cells

In [ ]:
# first, create a set of 0/1 mask files representing valid pixels
for input_path in raw_paths:
    
    # set the output file name
    output_path = '{file}?{options}'.format(file = input_path[:-4]+"-mask.tif", options = otb_creation_options)
    
    # build the otb command
    manageNoData.SetParameterString("in", input_path)
    manageNoData.SetParameterString("out", output_path)
    manageNoData.SetParameterString("mode", "buildmask")
    manageNoData.SetParameterValue("usenan", True)
    masking.SetParameterOutputImagePixelType("out", 1)
    
    # run it
    manageNoData.ExecuteAndWriteOutput()
    
    # sanitize the otb call for future use
    manageNoData.ClearValue("in")
    manageNoData.ClearValue("out")
    manageNoData.ClearValue("mode")
    manageNoData.ClearValue("mode.changevalue.newv")
    manageNoData.ClearValue("usenan")

In [40]:
# loop through each grid size and resample / reproject this raw data
for grid_size, directory in zip(config['data'], grid_dirs):
    print("Processing {key} data at grid size: {gs:06d} meters".format(key=key, gs=grid_size))

'/external/aedes-americas/data-mx-in/100000-m/pop.tif'